In [2]:
# %pip install pyspark==3.4.1

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
# from pyspark.sql.functions import array, col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import datetime

In [4]:
import os
cwd = os.getcwd()

In [5]:
# spark = SparkSession.builder.master("local[1]") \
#                     .appName('Price') \
#                     .getOrCreate()
spark = SparkSession.builder \
                    .master("spark://127.0.0.1:7077") \
                    .appName('Price') \
                    .getOrCreate()

In [6]:
p_df = spark.read.option("header", True)\
    .csv(f"{cwd}/dataset/Prices_E_All_Data/Prices_E_All_Data.csv")
# p_df.printSchema()

In [7]:
col_name = []
for col in p_df.dtypes:
    col_name.append(col[0])
# col_name

In [8]:
year_list = []
for x in col_name:
    if 'Y' in x and 'F' not in x:
        year_list.append(int(x.replace('Y', '')))
# year_list

In [9]:
ElementPrice_df = p_df.select(
    p_df['Element Code'].cast('int').alias('id'),
    p_df['Element'].cast('string').alias('element'),
    p_df['Unit'].cast('string').alias('unit'),
).orderBy('id')
ElementPrice_df = ElementPrice_df.dropDuplicates(["id"])
ElementPrice_df.show()

+----+--------------------+----+
|  id|             element|unit|
+----+--------------------+----+
|5530|Producer Price (L...| LCU|
|5531|Producer Price (S...| SLC|
|5532|Producer Price (U...| USD|
|5539|Producer Price In...|null|
+----+--------------------+----+



In [10]:
Month_df = p_df.select(
    p_df['Months Code'].cast('int').alias('id'),
    p_df['Months'].cast('string').alias('month'),
).orderBy('id')
Month_df = Month_df.dropDuplicates(["id"])
Month_df.show()

+----+------------+
|  id|       month|
+----+------------+
|7001|     January|
|7002|    February|
|7003|       March|
|7004|       April|
|7005|         May|
|7006|        June|
|7007|        July|
|7008|      August|
|7009|   September|
|7010|     October|
|7011|    November|
|7012|    December|
|7021|Annual value|
+----+------------+



In [11]:
for year in year_list:
    p_df = p_df.withColumn(f"NewColumn_{year}", F.array(f"Y{year}", f"Y{year}F", F.lit(year)))
    p_df = p_df.drop(f"Y{year}", f"Y{year}F")
    
p_df = p_df.withColumn(f"merge_column", F.array(*[f"NewColumn_{year}" for year in year_list]))
transformed_date = datetime.datetime.now()
for year in year_list:
    p_df = p_df.drop(f"NewColumn_{year}")
p_df = p_df.select(
    p_df['Area Code'].cast('int').alias('country_id'),
    p_df['Item Code'].cast('int').alias('item_prod_id'),
    p_df['Element Code'].cast('int').alias('element_price_id'),
    p_df['Months Code'].cast('int').alias('month_id'),
    F.explode('merge_column').alias('year_col')
)
p_df = p_df.withColumn('value', F.lit(p_df.year_col[0])) \
            .withColumn('flag', F.lit(p_df.year_col[1])) \
            .withColumn('year', F.lit(p_df.year_col[2])) \
            .withColumn('transformed_date', F.lit(transformed_date))
p_df = p_df.drop('year_col')
p_df.show()

+----------+------------+----------------+--------+-------------+----+----+--------------------+
|country_id|item_prod_id|element_price_id|month_id|        value|flag|year|    transformed_date|
+----------+------------+----------------+--------+-------------+----+----+--------------------+
|         2|         221|            5530|    7021|         null|null|1991|2023-07-26 22:58:...|
|         2|         221|            5530|    7021|         null|null|1992|2023-07-26 22:58:...|
|         2|         221|            5530|    7021| 46000.000000|   A|1993|2023-07-26 22:58:...|
|         2|         221|            5530|    7021| 50000.000000|   A|1994|2023-07-26 22:58:...|
|         2|         221|            5530|    7021| 62000.000000|   A|1995|2023-07-26 22:58:...|
|         2|         221|            5530|    7021| 50000.000000|   A|1996|2023-07-26 22:58:...|
|         2|         221|            5530|    7021| 41000.000000|   A|1997|2023-07-26 22:58:...|
|         2|         221|     

In [12]:
ElementPrice_df.write\
    .mode("overwrite") \
    .parquet("hdfs://127.0.0.1:9000/FAOSTAT_prj/DataWarehouse/Element_Price")

In [13]:
Month_df.write\
    .mode("overwrite") \
    .parquet("hdfs://127.0.0.1:9000/FAOSTAT_prj/DataWarehouse/Month")

In [14]:
p_df.write\
    .mode("overwrite") \
    .parquet("hdfs://127.0.0.1:9000/FAOSTAT_prj/DataWarehouse/Price")

In [15]:
spark.stop()